In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchmetrics
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as f
import os

import matplotlib.pyplot as plt
import numpy as np

In [13]:
transform = transforms.Compose([transforms.ToTensor()])

train_batch_size = 50
test_batch_size = 20
train_set = torchvision.datasets.MNIST(root='./data/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.MNIST(root='./data/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=False, num_workers=2)

In [14]:
torch.manual_seed(1)
plt.rcParams["savefig.bbox"] = 'tight'


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = f.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [15]:
class LeNet(nn.Module):
    def __init__(self, input_channels: int, num_classe: int):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=6, kernel_size=5, padding=2)
        self.avg_pol1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avg_pol2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=5*5*16, out_features=120)
        self.fc2 =  nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classe)
        self.soft = nn.Softmax(dim=1)

    def forward(self, X):
        X = self.conv1(X)
        X = F.relu(X)

        X = self.avg_pol1(X)

        X = self.conv2(X)
        X = F.relu(X)

        X = self.avg_pol2(X)

        X = X.view(-1, 16*5*5) # flatten all conv features

        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)

        return self.soft(X)


In [16]:
# training
from torch.optim import RMSprop, Adam
from torch.nn import CrossEntropyLoss

model = LeNet(input_channels=1, num_classe=10)
optimizer = Adam(model.parameters(), lr=0.001)
cross_entropy = CrossEntropyLoss()

epochs = 2
for _ in range(epochs):
    for i, (x_train, y_train) in enumerate(train_loader):
        y_pred = model(x_train)
        loss = cross_entropy(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Accuracy ->  tensor(0.0400)
Accuracy ->  tensor(0.1200)
Accuracy ->  tensor(0.1000)
Accuracy ->  tensor(0.0600)
Accuracy ->  tensor(0.1200)
Accuracy ->  tensor(0.1200)
Accuracy ->  tensor(0.1800)
Accuracy ->  tensor(0.2600)
Accuracy ->  tensor(0.2600)
Accuracy ->  tensor(0.4600)
Accuracy ->  tensor(0.3800)
Accuracy ->  tensor(0.4000)
Accuracy ->  tensor(0.3800)
Accuracy ->  tensor(0.6000)
Accuracy ->  tensor(0.4400)
Accuracy ->  tensor(0.4600)
Accuracy ->  tensor(0.4000)
Accuracy ->  tensor(0.2800)
Accuracy ->  tensor(0.4400)
Accuracy ->  tensor(0.4000)
Accuracy ->  tensor(0.3000)
Accuracy ->  tensor(0.3000)
Accuracy ->  tensor(0.2600)
Accuracy ->  tensor(0.3200)
Accuracy ->  tensor(0.4000)
Accuracy ->  tensor(0.3600)
Accuracy ->  tensor(0.3200)
Accuracy ->  tensor(0.3200)
Accuracy ->  tensor(0.4800)
Accuracy ->  tensor(0.5200)
Accuracy ->  tensor(0.4400)
Accuracy ->  tensor(0.4200)
Accuracy ->  tensor(0.4400)
Accuracy ->  tensor(0.5400)
Accuracy ->  tensor(0.5800)
Accuracy ->  tensor(

In [26]:
# test
accuracy = Accuracy(task="multiclass", num_classes=10)

for i, (x_test, y_test) in enumerate(test_loader):
    y_pred = model(x_test)
    prediction = torch.argmax(y_pred, dim=1)
    real_value = y_test
    print('Real -> ', real_value)
    print('predicted ->', prediction, '\n\n')
    print(f'Accuracy of {i} batch', accuracy(y_pred, y_test))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [25]:
# training on GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LeNet(input_channels=1, num_classe=10)
optimizer = Adam(model.parameters(), lr=0.001)
cross_entropy = CrossEntropyLoss()

model.to(device)
epochs = 2
for _ in range(epochs):
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        y_pred = model(x_train)
        loss = cross_entropy(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()